In [38]:
import http.client
import urllib.parse
import pandas as pd
import json
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)



def processData(apiKey, blockchain, network, address, beginTime=None, endTime=None, timezone=None):
    # Base URL and API endpoint
    base_url = "rest.cryptoapis.io"
    endpoint = f"/blockchain-data/{blockchain}/{network}/addresses/{address}/transactions"

    # Construct query parameters
    query_params = {
        "limit": 50,  # Fetch 50 transactions per request
        "offset": 0,  # Starting offset
    }

    # Encode the query parameters
    querystring = f"{endpoint}?{urllib.parse.urlencode(query_params)}"
    # print(querystring)
    # Configuring headers
    headers = {
        'Content-Type': "application/json",
        'x-api-key': apiKey  # Replace with your actual API key
    }

    # Create a connection to the server
    conn = http.client.HTTPSConnection(base_url)

    # Send GET request
    conn.request("GET", querystring, headers=headers)

    # Get and read response
    res = conn.getresponse()
    data = res.read()

    # Decoding the response data
    response_data = data.decode("utf-8")
    # Convert the JSON response to a dictionary
    response_dict = json.loads(response_data)

    # Extract the transactions data
    transactions = response_dict.get('data', {}).get('items', [])

    # Convert the transactions data to a DataFrame
    df = pd.json_normalize(transactions)
    
    if not df.empty:
        df['recipients.address'] = df['recipients'].apply(lambda x: x[0]['address'])
        df['recipients.amount'] = df['recipients'].apply(lambda x: x[0]['amount'])
        df['senders.address'] = df['senders'].apply(lambda x: x[0]['address'])
        df['senders.amount'] = df['senders'].apply(lambda x: x[0]['amount'])

        df.rename(columns={'unit': 'amount', 
                               'transactionHash': 'txHash', 'timestamp': 'datetime'}, inplace=True) 
        #df['datetime'] = pd.to_datetime(df['datetime'], unit='s').dt.strftime('%Y-%m-%dT%H:%M:%SZ')
        df['contactPlatformSlug'] = ''
        df['direction'] = df['recipients.address'].apply(lambda x: 'IN' if x == address else 'OUT')
        df['currency'] = ''
        df['amount'] = df['amount'].astype(float)
        df['contactIdentity'] = df.apply(lambda row: row['recipients.address'] if row['senders.address'] == address else row['recipients.address'], axis=1)
        df['type'] = df.apply(lambda row: 'CHAIN_TRANSFER_IN' if row['recipients.address'] == address else 'CHAIN_TRANSFER_OUT', axis=1)
        
        df = df[['type', 'txHash', 'datetime', 'contactIdentity',
                      'contactPlatformSlug', 'direction', 'currency', 'amount']]

        # 过滤重复行
        df = df.drop_duplicates(subset=['txHash', 'contactIdentity', 'amount'])
        df.reset_index(drop=True, inplace=True)
        
    return df


blockchain = "base"
network = "mainnet"
address = "0x9b6899D37D3200a9EAe9F0E24765E8cC2057856D"
#beginTime = "2023-01-01T00:00:00Z"
#endTime = "2023-01-31T23:59:59Z"
apiKey = "0346fea2811f3f7f81aa351e82d16e514b636282"
#timezone = "UTC"

processData(apiKey, blockchain, network, address, beginTime, endTime, timezone)

,type,txHash,datetime,contactIdentity,contactPlatformSlug,direction,currency,amount
0,CHAIN_TRANSFER_OUT,0xeda211d3332af0814c9f335a36b65d266e0fffa3d525...,+056490-05-09T03:36:40.000Z,0x8f44fd754285aa6a2b8b9b97739b79746e0475a7,,OUT,,0.000000
1,CHAIN_TRANSFER_OUT,0x3a7b87272aca04e830cf2cdddc4de7959644f747bc2c...,+056490-05-09T11:23:20.000Z,0x1195cf65f83b3a5768f3c496d3a05ad6412c64b7,,OUT,,0.000075
2,CHAIN_TRANSFER_OUT,0xb7fe7a3d04c942694c11115ebabd24e2c08189ea9550...,+056490-05-14T15:50:00.000Z,0x1195cf65f83b3a5768f3c496d3a05ad6412c64b7,,OUT,,0.000075
3,CHAIN_TRANSFER_OUT,0x9013ef489346b03f0f33f7145924b91eb1b58afa1361...,+056490-05-17T21:36:40.000Z,0x1195cf65f83b3a5768f3c496d3a05ad6412c64b7,,OUT,,0.000075
4,CHAIN_TRANSFER_OUT,0xff63797c07a814af2af42a05a53983f216a216db5fa3...,+056490-05-21T15:03:20.000Z,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,,OUT,,0.000010
5,CHAIN_TRANSFER_OUT,0x072bf1cb0b11d7a8889b14b6a3bb789e81a193627080...,+056490-05-21T20:03:20.000Z,0x1195cf65f83b3a5768f3c496d3a05ad6412c64b7,,OUT,,0.000075
6,CHAIN_TRANSFER_OUT,0xa92cdbea74a8113c6a095883cf005936d82513e9cae5...,+056490-05-22T03:16:40.000Z,0xa70e4d313fba81280137483a1b6d8f738db77a63,,OUT,,0.000000
